In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import audio_dataset_from_directory
import numpy as np
import librosa

#tf.compat.v1.disable_eager_execution()
#import tensorflow_io as tfio

In [ ]:
#Getting the input data from drive (eins, zwei, drei folders)
!gdown 10P678fWDyAJIRv_HlqsXtS2u68NTFZ7I
!unzip data_cnn.zip

Downloading...
From (original): https://drive.google.com/uc?id=10P678fWDyAJIRv_HlqsXtS2u68NTFZ7I
From (redirected): https://drive.google.com/uc?id=10P678fWDyAJIRv_HlqsXtS2u68NTFZ7I&confirm=t&uuid=22c5dcb2-3a2d-4047-acab-2cfc4c0e4745
To: /content/data_cnn.zip
100% 118M/118M [00:05<00:00, 23.3MB/s]
Archive:  data_cnn.zip
   creating: data_cnn/drei/
  inflating: data_cnn/drei/Q1200603.wav  
  inflating: data_cnn/drei/Q1200703.wav  
  inflating: data_cnn/drei/Q1200803.wav  
  inflating: data_cnn/drei/Q1200903.wav  
  inflating: data_cnn/drei/Q1201103.wav  
  inflating: data_cnn/drei/Q1201203.wav  
  inflating: data_cnn/drei/Q1201303.wav  
  inflating: data_cnn/drei/Q1201503.wav  
  inflating: data_cnn/drei/Q1201703.wav  
  inflating: data_cnn/drei/Q1201803.wav  
  inflating: data_cnn/drei/Q1201903.wav  
  inflating: data_cnn/drei/Q1202003.wav  
  inflating: data_cnn/drei/Q1202103.wav  
  inflating: data_cnn/drei/Q1202203.wav  
  inflating: data_cnn/drei/Q1202303.wav  
  inflating: data_cnn

In [ ]:

data_dir = "/content/data_cnn"

train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=None,
    validation_split=0.2,
    seed=0,
    subset='both')

Found 526 files belonging to 3 classes.
Using 421 files for training.
Using 105 files for validation.


In [ ]:
def preprocess_audio_mfps(audio, label):
    # Convert audio tensor to a compatible format
    audio = tf.cast(audio, tf.float32)  # Cast audio to float32
    audio = audio / 32768.0  # Normalize audio

    # Extract mel-frequency power spectra
    def _extract_mel(audio):
        # Reshape the audio tensor to (batch_size, num_samples) as expected by tf.signal.stft
        audio = tf.reshape(audio, [-1])

        # Compute mel-frequency power spectra
        stfts = tf.signal.stft(audio, frame_length=1024, frame_step=512, fft_length=1024)
        spectrograms = tf.abs(stfts)

        num_spectrogram_bins = stfts.shape[-1]
        lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, 128
        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins, num_spectrogram_bins, 16000, lower_edge_hertz, upper_edge_hertz)

        mel_spectrograms = tf.tensordot(spectrograms, linear_to_mel_weight_matrix, 1)

        return mel_spectrograms

    # Use tf.py_function to call _extract_mel with audio tensor
    mel_spectra = tf.py_function(_extract_mel, [audio], tf.float32)

    return mel_spectra, label

In [ ]:
train_processed_mspec = train_ds.map(preprocess_audio_mfps)
val_processed_mspec = val_ds.map(preprocess_audio_mfps)

In [ ]:

# Find the maximum sequence length in the training dataset
max_length = max(len(seq) for seq, _ in train_processed_mspec.as_numpy_iterator())

# Function to pad sequences
def pad_sequence(seq, label):
    padded_seq = tf.pad(seq, paddings=[[0, max_length - tf.shape(seq)[0]], [0, 0]])
    return padded_seq, label

# Pad the training dataset
padded_train_ds_mfps = train_processed_mspec.map(pad_sequence)

# Pad the validation dataset
padded_val_ds_mfps = val_processed_mspec.map(pad_sequence)

In [ ]:
import numpy as np

def create_pairs_and_labels(padded_dataset):
    pairs = []
    labels = []

    for sequence, label in padded_dataset:
        # Assuming 'sequence' is your padded sequence and 'label' is its corresponding label
        pairs.append(sequence)
        labels.append(label)

    # Convert lists to numpy arrays
    pairs = np.array(pairs)
    labels = np.array(labels)

    return pairs, labels

# Apply the function to your padded datasets
train_data, train_labels = create_pairs_and_labels(padded_train_ds_mfps)
val_data, val_labels = create_pairs_and_labels(padded_val_ds_mfps)


In [ ]:
# Assuming you have your dataset X and corresponding labels y
# X.shape = (num_samples, input_vector_size)
# y.shape = (num_samples,)

# Function to create pairs of data and labels
def create_pairs(X, y, num_pairs):
    pairs = []
    labels = []
    num_classes = len(np.unique(y))
    class_indices = [np.where(y == i)[0] for i in range(num_classes)]

    for _ in range(num_pairs):
        # Select a random class (label)
        class_idx = np.random.randint(0, num_classes)
        # Select a random sample from the selected class
        idx_1 = np.random.choice(class_indices[class_idx])
        # Ensure that the second sample is from the same class for half of the pairs
        should_be_same_class = np.random.randint(0, 2)
        if should_be_same_class:
            idx_2 = np.random.choice(class_indices[class_idx])
        else:
            # Select a random class different from the first one
            class_idx_2 = (class_idx + np.random.randint(1, num_classes)) % num_classes
            idx_2 = np.random.choice(class_indices[class_idx_2])
        pairs.append([X[idx_1], X[idx_2]])
        # 1 if same class, 0 if different class
        labels.append(1 if should_be_same_class else 0)

    return np.array(pairs), np.array(labels)

# Example usage:
num_pairs = 1500  # Adjust this number based on your dataset size and requirements
train_pairs, train_pairs_labels = create_pairs(train_data, train_labels, num_pairs)
num_pairs_val = 500
val_pairs, val_pairs_labels = create_pairs(val_data, val_labels, num_pairs_val)

In [ ]:

# Convert labels to integers
train_pairs_labels = train_pairs_labels.astype(np.float32)
val_pairs_labels = val_pairs_labels.astype(np.float32)


In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, GlobalAveragePooling1D, Lambda
from keras.optimizers import Adam, SGD, RMSprop
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import keras.backend as K

# Define the Siamese Network model architecture with variable layers, filters, and kernel size
def siamese_model(params):
    input_shape = (402, 128)  # Define the shape of your input vectors
    model_input = Input(shape=input_shape)
    x = model_input

    # Dynamically adding layers based on num_layers
    for i in range(params['num_layers']):
        x = Conv1D(params['num_filters'], kernel_size=params['kernel_size'], activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        if i < params['num_layers'] - 1:  # add max pooling to all but the last layer
            x = MaxPooling1D(pool_size=2, strides=2)(x)

    x = GlobalAveragePooling1D()(x)
    return Model(inputs=model_input, outputs=x)

# Define cosine similarity function
def cosine_similarity(vectors):
    x, y = vectors
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return K.sum(x * y, axis=-1, keepdims=True)

# Define contrastive loss function
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    return K.mean(y_true * K.square(1 - y_pred) + (1 - y_true) * K.square(K.maximum(y_pred - margin, 0)))

# Define the objective function for hyperparameter optimization
def objective(params):
    # Create the Siamese model with current params
    base_model = siamese_model(params)

    # Inputs of the Siamese Network
    input_a = Input(shape=(402, 128))
    input_b = Input(shape=(402, 128))

    # Process both inputs through the same base model
    processed_a = base_model(input_a)
    processed_b = base_model(input_b)

    # Calculate the cosine similarity
    cosine_sim = Lambda(cosine_similarity, output_shape=(1,))([processed_a, processed_b])

    # Final Siamese network model
    siamese_network = Model(inputs=[input_a, input_b], outputs=cosine_sim)

    # Select and configure the optimizer
    if params['optimizer']['type'] == 'adam':
        optimizer = Adam(learning_rate=params['optimizer']['lr'])
    elif params['optimizer']['type'] == 'sgd':
        optimizer = SGD(learning_rate=params['optimizer']['lr'], momentum=params['optimizer']['momentum'])
    elif params['optimizer']['type'] == 'rmsprop':
        optimizer = RMSprop(learning_rate=params['optimizer']['lr'])

    siamese_network.compile(optimizer=optimizer, loss=contrastive_loss)

    # Dummy data placeholders, replace with actual data
    x1_train, x2_train, y_train, x1_val, x2_val, y_val =train_pairs[:, 0], train_pairs[:, 1], train_pairs_labels,val_pairs[:, 0], val_pairs[:, 1], val_pairs_labels

    # Training (assuming dummy placeholders are replaced)
    result = siamese_network.fit([x1_train, x2_train], y_train, epochs=10, batch_size=params['batch_size'], validation_data=([x1_val, x2_val], y_val), verbose=0)
    validation_loss = np.min(result.history['val_loss'])
    return {'loss': validation_loss, 'status': STATUS_OK}

# Define the search space for hyperparameters
space = {
    'num_layers': hp.choice('num_layers', range(3, 7)),  # Reduced range for simplicity
    'num_filters': hp.choice('num_filters', [16, 32, 64, 128, 256]),
    'kernel_size': hp.choice('kernel_size', [3, 5, 7]),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    'optimizer': hp.choice('optimizer', [
        {'type': 'adam', 'lr': hp.loguniform('adam_lr', np.log(1e-4), np.log(1e-2))},
        {'type': 'sgd', 'lr': hp.loguniform('sgd_lr', np.log(1e-4), np.log(1e-2)), 'momentum': hp.uniform('sgd_momentum', 0.0, 0.99)},
        {'type': 'rmsprop', 'lr': hp.loguniform('rmsprop_lr', np.log(1e-4), np.log(1e-2))}
    ])
}

# Running the hyperparameter optimization
trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,  # Adjust based on computational resources and needs
    trials=trials
)

print("Best parameters:", best_params)


100%|██████████| 10/10 [04:08<00:00, 24.82s/trial, best loss: 1.4637180373598723e-15]
Best parameters: {'batch_size': 1, 'kernel_size': 1, 'num_filters': 3, 'num_layers': 2, 'optimizer': 1, 'sgd_lr': 0.00015588365602147455, 'sgd_momentum': 0.8494444660753903}


In [ ]:
# After finding the best parameters, decode them to their actual meanings
def decode_hyperparameters(params):
    return {
        'num_layers': range(3, 7)[params['num_layers']],
        'num_filters': [16, 32, 64, 128, 256, 512][params['num_filters']],
        'kernel_size': [3, 5, 7][params['kernel_size']],
        'batch_size': [16, 32, 64][params['batch_size']]
        #'optimizer': ['adam', 'sgd', 'rmsprop'][params['optimizer']['type']],'lr': params['optimizer']['lr']  # This will need proper mapping if it's also indexed
    }

decoded_params = decode_hyperparameters(best_params)
print("Decoded parameters:", decoded_params)


Decoded parameters: {'num_layers': 5, 'num_filters': 128, 'kernel_size': 5, 'batch_size': 32}


In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, GlobalAveragePooling1D, Lambda
from keras.optimizers import Adam, SGD, RMSprop
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import keras.backend as K

# Define the Siamese Network model architecture with variable layers, filters, and kernel size
def siamese_model(params):
    input_shape = (402, 128)  # Define the shape of your input vectors
    model_input = Input(shape=input_shape)
    x = model_input

    # Dynamically adding layers based on num_layers
    for i in range(params['num_layers']):
        x = Conv1D(params['num_filters'], kernel_size=params['kernel_size'], activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        if i < params['num_layers'] - 1:  # add max pooling to all but the last layer
            x = MaxPooling1D(pool_size=2, strides=2)(x)

    x = GlobalAveragePooling1D()(x)
    return Model(inputs=model_input, outputs=x)

# Define cosine similarity function
def cosine_similarity(vectors):
    x, y = vectors
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return K.sum(x * y, axis=-1, keepdims=True)

# Define contrastive loss function
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    return K.mean(y_true * K.square(1 - y_pred) + (1 - y_true) * K.square(K.maximum(y_pred - margin, 0)))

# Define the objective function for hyperparameter optimization
def objective(params):
    # Create the Siamese model with current params
    base_model = siamese_model(params)

    # Inputs of the Siamese Network
    input_a = Input(shape=(402, 128))
    input_b = Input(shape=(402, 128))

    # Process both inputs through the same base model
    processed_a = base_model(input_a)
    processed_b = base_model(input_b)

    # Calculate the cosine similarity
    cosine_sim = Lambda(cosine_similarity, output_shape=(1,))([processed_a, processed_b])

    # Final Siamese network model
    siamese_network = Model(inputs=[input_a, input_b], outputs=cosine_sim)

    # Select and configure the optimizer
    if params['optimizer']['type'] == 'adam':
        optimizer = Adam(learning_rate=params['optimizer']['lr'])
    elif params['optimizer']['type'] == 'sgd':
        optimizer = SGD(learning_rate=params['optimizer']['lr'], momentum=params['optimizer']['momentum'])
    elif params['optimizer']['type'] == 'rmsprop':
        optimizer = RMSprop(learning_rate=params['optimizer']['lr'])

    siamese_network.compile(optimizer=optimizer, loss=contrastive_loss)

    # Dummy data placeholders, replace with actual data
    x1_train, x2_train, y_train, x1_val, x2_val, y_val =train_pairs[:, 0], train_pairs[:, 1], train_pairs_labels,val_pairs[:, 0], val_pairs[:, 1], val_pairs_labels

    # Training (assuming dummy placeholders are replaced)
    result = siamese_network.fit([x1_train, x2_train], y_train, epochs=10, batch_size=params['batch_size'], validation_data=([x1_val, x2_val], y_val), verbose=0)
    validation_loss = np.min(result.history['val_loss'])
    return {'loss': validation_loss, 'status': STATUS_OK}

# Define the search space for hyperparameters
space = {
    'num_layers': hp.choice('num_layers', range(3, 7)),  # Reduced range for simplicity
    'num_filters': hp.choice('num_filters', [16, 32, 64, 128, 256]),
    'kernel_size': hp.choice('kernel_size', [3, 5, 7]),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    'optimizer': hp.choice('optimizer', [
        {'type': 'adam', 'lr': hp.loguniform('adam_lr', np.log(1e-4), np.log(1e-2))},
        {'type': 'sgd', 'lr': hp.loguniform('sgd_lr', np.log(1e-4), np.log(1e-2)), 'momentum': hp.uniform('sgd_momentum', 0.0, 0.99)},
        {'type': 'rmsprop', 'lr': hp.loguniform('rmsprop_lr', np.log(1e-4), np.log(1e-2))}
    ])
}

# Running the hyperparameter optimization
trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=25,  # Adjust based on computational resources and needs
    trials=trials
)

print("Best parameters:", best_params)


100%|██████████| 25/25 [09:33<00:00, 22.94s/trial, best loss: 1.8900436957565913e-15]
Best parameters: {'batch_size': 2, 'kernel_size': 2, 'num_filters': 2, 'num_layers': 3, 'optimizer': 1, 'sgd_lr': 0.00010112553577170664, 'sgd_momentum': 0.308640937804029}


In [ ]:
# After finding the best parameters, decode them to their actual meanings
def decode_hyperparameters(params):
    return {
        'num_layers': range(3, 7)[params['num_layers']],
        'num_filters': [16, 32, 64, 128, 256, 512][params['num_filters']],
        'kernel_size': [3, 5, 7][params['kernel_size']],
        'batch_size': [16, 32, 64][params['batch_size']],
        #'optimizer': ['adam', 'sgd', 'rmsprop'][params['optimizer']['type']],
       # 'lr': params['optimizer']['lr']  # This will need proper mapping if it's also indexed
    }

decoded_params = decode_hyperparameters(best_params)
print("Decoded parameters:", decoded_params)


Decoded parameters: {'num_layers': 6, 'num_filters': 64, 'kernel_size': 7, 'batch_size': 64}


In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, GlobalAveragePooling1D, Lambda
from keras.optimizers import Adam, SGD, RMSprop
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import keras.backend as K

# Define the Siamese Network model architecture with variable layers, filters, and kernel size
def siamese_model(params):
    input_shape = (402, 128)  # Define the shape of your input vectors
    model_input = Input(shape=input_shape)
    x = model_input

    # Dynamically adding layers based on num_layers
    for i in range(params['num_layers']):
        x = Conv1D(params['num_filters'], kernel_size=params['kernel_size'], activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        if i < params['num_layers'] - 1:  # add max pooling to all but the last layer
            x = MaxPooling1D(pool_size=2, strides=2)(x)

    x = GlobalAveragePooling1D()(x)
    return Model(inputs=model_input, outputs=x)

# Define cosine similarity function
def cosine_similarity(vectors):
    x, y = vectors
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return K.sum(x * y, axis=-1, keepdims=True)

# Define contrastive loss function
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    return K.mean(y_true * K.square(1 - y_pred) + (1 - y_true) * K.square(K.maximum(y_pred - margin, 0)))

# Define the objective function for hyperparameter optimization
def objective(params):
    # Create the Siamese model with current params
    base_model = siamese_model(params)

    # Inputs of the Siamese Network
    input_a = Input(shape=(402, 128))
    input_b = Input(shape=(402, 128))

    # Process both inputs through the same base model
    processed_a = base_model(input_a)
    processed_b = base_model(input_b)

    # Calculate the cosine similarity
    cosine_sim = Lambda(cosine_similarity, output_shape=(1,))([processed_a, processed_b])

    # Final Siamese network model
    siamese_network = Model(inputs=[input_a, input_b], outputs=cosine_sim)

    # Select and configure the optimizer
    if params['optimizer']['type'] == 'adam':
        optimizer = Adam(learning_rate=params['optimizer']['lr'])
    elif params['optimizer']['type'] == 'sgd':
        optimizer = SGD(learning_rate=params['optimizer']['lr'], momentum=params['optimizer']['momentum'])
    elif params['optimizer']['type'] == 'rmsprop':
        optimizer = RMSprop(learning_rate=params['optimizer']['lr'])

    siamese_network.compile(optimizer=optimizer, loss=contrastive_loss)

    # Dummy data placeholders, replace with actual data
    x1_train, x2_train, y_train, x1_val, x2_val, y_val =train_pairs[:, 0], train_pairs[:, 1], train_pairs_labels,val_pairs[:, 0], val_pairs[:, 1], val_pairs_labels

    # Training (assuming dummy placeholders are replaced)
    result = siamese_network.fit([x1_train, x2_train], y_train, epochs=10, batch_size=params['batch_size'], validation_data=([x1_val, x2_val], y_val), verbose=0)
    validation_loss = np.min(result.history['val_loss'])
    return {'loss': validation_loss, 'status': STATUS_OK}

# Define the search space for hyperparameters
space = {
    'num_layers': hp.choice('num_layers', range(3, 7)),  # Reduced range for simplicity
    'num_filters': hp.choice('num_filters', [16, 32, 64, 128, 256]),
    'kernel_size': hp.choice('kernel_size', [3, 5, 7]),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    'optimizer': hp.choice('optimizer', [
        {'type': 'adam', 'lr': hp.loguniform('adam_lr', np.log(1e-4), np.log(1e-2))},
        {'type': 'sgd', 'lr': hp.loguniform('sgd_lr', np.log(1e-4), np.log(1e-2)), 'momentum': hp.uniform('sgd_momentum', 0.0, 0.99)},
        {'type': 'rmsprop', 'lr': hp.loguniform('rmsprop_lr', np.log(1e-4), np.log(1e-2))}
    ])
}

# Running the hyperparameter optimization
trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,  # Adjust based on computational resources and needs
    trials=trials
)

print("Best parameters:", best_params)


 54%|█████▍    | 27/50 [10:48<07:33, 19.72s/trial, best loss: 1.2576606499186739e-15]

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, GlobalAveragePooling1D, Lambda
from keras.optimizers import Adam, SGD, RMSprop
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import keras.backend as K

# Define the Siamese Network model architecture with variable layers, filters, and kernel size
def siamese_model(params):
    input_shape = (402, 128)  # Define the shape of your input vectors
    model_input = Input(shape=input_shape)
    x = model_input

    # Dynamically adding layers based on num_layers
    for i in range(params['num_layers']):
        x = Conv1D(params['num_filters'], kernel_size=params['kernel_size'], activation=params['activation'], padding='same')(x)
        x = BatchNormalization()(x)
        if i < params['num_layers'] - 1:  # add max pooling to all but the last layer
            x = MaxPooling1D(pool_size=2, strides=2)(x)

    x = GlobalAveragePooling1D()(x)
    return Model(inputs=model_input, outputs=x)

# Define cosine similarity function
def cosine_similarity(vectors):
    x, y = vectors
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return K.sum(x * y, axis=-1, keepdims=True)

# Define contrastive loss function
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    return K.mean(y_true * K.square(1 - y_pred) + (1 - y_true) * K.square(K.maximum(y_pred - margin, 0)))

# Define the objective function for hyperparameter optimization
def objective(params):
    # Create the Siamese model with current params
    base_model = siamese_model(params)

    # Inputs of the Siamese Network
    input_a = Input(shape=(402, 128))
    input_b = Input(shape=(402, 128))

    # Process both inputs through the same base model
    processed_a = base_model(input_a)
    processed_b = base_model(input_b)

    # Calculate the cosine similarity
    cosine_sim = Lambda(cosine_similarity, output_shape=(1,))([processed_a, processed_b])

    # Final Siamese network model
    siamese_network = Model(inputs=[input_a, input_b], outputs=cosine_sim)

    # Select and configure the optimizer
    if params['optimizer']['type'] == 'adam':
        optimizer = Adam(learning_rate=params['optimizer']['lr'])
    elif params['optimizer']['type'] == 'sgd':
        optimizer = SGD(learning_rate=params['optimizer']['lr'], momentum=params['optimizer']['momentum'])
    elif params['optimizer']['type'] == 'rmsprop':
        optimizer = RMSprop(learning_rate=params['optimizer']['lr'])

    siamese_network.compile(optimizer=optimizer, loss=contrastive_loss)

    # Dummy data placeholders, replace with actual data
    x1_train, x2_train, y_train, x1_val, x2_val, y_val =train_pairs[:, 0], train_pairs[:, 1], train_pairs_labels,val_pairs[:, 0], val_pairs[:, 1], val_pairs_labels

    # Training (assuming dummy placeholders are replaced)
    result = siamese_network.fit([x1_train, x2_train], y_train, epochs=10, batch_size=params['batch_size'], validation_data=([x1_val, x2_val], y_val), verbose=0)
    validation_loss = np.min(result.history['val_loss'])
    return {'loss': validation_loss, 'status': STATUS_OK}

# Define the search space for hyperparameters
space = {
    'num_layers': hp.choice('num_layers', range(3, 7)),  # Reduced range for simplicity
    'num_filters': hp.choice('num_filters', [16, 32, 64, 128, 256]),
    'kernel_size': hp.choice('kernel_size', [3, 5, 7]),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    'activation': hp.choice('activation', ['relu', 'tanh', 'sigmoid']),
    'optimizer': hp.choice('optimizer', [
        {'type': 'adam', 'lr': hp.loguniform('adam_lr', np.log(1e-4), np.log(1e-2))},
        {'type': 'sgd', 'lr': hp.loguniform('sgd_lr', np.log(1e-4), np.log(1e-2)), 'momentum': hp.uniform('sgd_momentum', 0.0, 0.99)},
        {'type': 'rmsprop', 'lr': hp.loguniform('rmsprop_lr', np.log(1e-4), np.log(1e-2))}
    ])
}

# Running the hyperparameter optimization
trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,  # Adjust based on computational resources and needs
    trials=trials
)

print("Best parameters:", best_params)



100%|██████████| 10/10 [21:42<00:00, 130.26s/trial, best loss: 1.570299451959052e-15]
Best parameters: {'activation': 2, 'adam_lr': 0.0006009855440680273, 'batch_size': 1, 'kernel_size': 0, 'num_filters': 2, 'num_layers': 2, 'optimizer': 0}


In [ ]:
# After finding the best parameters, decode them to their actual meanings
def decode_hyperparameters(params):
    return {
        'num_layers': range(3, 7)[params['num_layers']],
        'num_filters': [16, 32, 64, 128, 256, 512][params['num_filters']],
        'kernel_size': [3, 5, 7][params['kernel_size']],
        'batch_size': [16, 32, 64][params['batch_size']],
        #'optimizer': ['adam', 'sgd', 'rmsprop'][params['optimizer']['type']],
       # 'lr': params['optimizer']['lr']  # This will need proper mapping if it's also indexed
    }

decoded_params = decode_hyperparameters(best_params)
print("Decoded parameters:", decoded_params)


Decoded parameters: {'num_layers': 5, 'num_filters': 64, 'kernel_size': 3, 'batch_size': 32}
